# Part 2 


In [27]:
try:
    import os
    import sys
    import numpy as np
    import elasticsearch
    from elasticsearch import Elasticsearch 
    import pandas as pd
    from elasticsearch import helpers
    print("All Modules Loaded ! ")
except Exception as e:
    print("Some Modules are Missing {}".format(e))

All Modules Loaded ! 


### Elasticsearch Setup

In [28]:
ENDPOINT = "http://localhost:9200/"
es = Elasticsearch(timeout=600,hosts=ENDPOINT)

In [29]:
es.ping()

True

### Dataset Preparation

In [31]:
df = pd.read_csv("BX-Books.csv")

In [32]:
rdf = pd.read_csv("BX-Book-Ratings.csv")

In [33]:
rdf.head()

,uid,isbn,rating
0,2,0195153448,0
1,8,0002005018,5
2,11400,0002005018,0
3,11676,0002005018,8
4,41385,0002005018,0


In [37]:
rdf.uid.value_counts()

11676     11144
198711     6456
153662     5814
98391      5779
35859      5646
          ...  
115357        1
17201         1
117404        1
31542         1
145222        1
Name: uid, Length: 92107, dtype: int64

### User Input

##### We Found that the User " 11676" has the most rated books in the user rating dataset

In [38]:
 user = int(input("Input User Id :")) 

Input User Id :11676


#### Group Books and get Mean user Rating

In [10]:
new = rdf.groupby('isbn',as_index= False).rating.mean() 
new.head()

,isbn,rating
0,0000913154,8.0
1,0001010565,0.0
2,0001046438,9.0
3,0001046713,0.0
4,000104687X,6.0


#### "user_r" is the dataset with the chosen user's ratings

In [11]:
user_r = rdf[rdf['uid']== user]
user_r = user_r.drop(['uid'], axis=1)
user_r = user_r.rename({'rating': 'user_rating'}, axis=1)

user_r.shape

(11144, 2)

In [12]:
"""user_r = user_r = pd.merge(user_r,new, on="isbn", how="inner", validate="one_to_one") # Exei kanei kai reviews se vivlia pou den yparxoun sto arxiko df

user_r = user_r.drop(['rating'], axis=1)"""
user_r.shape

(11144, 2)

#### "result' is the dataset with all the books and their information and also their mean user rating

In [13]:
result = pd.merge(df, new, on="isbn", how="inner", validate="one_to_one")

In [14]:
result.shape

(134692, 8)

In [15]:
result.shape

(134692, 8)

#### "final" is the dataset with the books and their information. Ther books here are the ones that the user we choose has rated or has read . We assume that the books in the user rating dataset with user_rating == 0  are books that the user has read and has not yet rated.

In [16]:
final= pd.merge(result,user_r, on="isbn", how="left", validate="one_to_one")
final=final.fillna(0)
final.head()

,isbn,book_title,book_author,year_of_publication,publisher,summary,category,rating,user_rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,Provides an introduction to classical myths pl...,['social science'],0.000000,0.0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",['actresses'],4.928571,8.0
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,"Here, for the first time in paperback, is an o...",['1940-1949'],5.000000,0.0
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an o...",['medical'],4.272727,0.0
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,A look at the incredibly well-preserved ancien...,['design'],0.000000,0.0


In [17]:
result.shape

(134692, 8)

In [18]:
user_r.shape

(11144, 2)

In [19]:
final.shape

(134692, 9)

## Prepare the data for elasticsearch. The steps below are also described in Part 1.

In [20]:
result2 = final.to_dict('records')

In [21]:
def generator(result2):
    for c, line in enumerate(result2):
        yield{
            '_index':'rbooks_elk',
            '_type':'_doc',
            '_id':line.get("isbn",None),
            '_source':{
                "title":line.get("book_title",""),
                "author":line.get("book_author",""),
                "summary":line.get("summary",""),
                "category":line.get("category",""),
                "year":line.get("year_of_publication",None),
                "publisher":line.get("publisher",""),
                "rating":line.get("rating",None),
                "user_rating":line.get("user_rating",None)
                
            }        
    }
    raise StopIteration

In [22]:
custom = generator(result2)

In [23]:
len(result2)

134692

In [24]:

next(custom)

{'_index': 'rbooks_elk',
 '_type': '_doc',
 '_id': '0195153448',
 '_source': {'title': 'Classical Mythology',
  'author': 'Mark P. O. Morford',
  'summary': 'Provides an introduction to classical myths placing the addressed\ntopics within their historical context, discussion of archaeological\nevidence as support for mythical events, and how these themes have\nbeen portrayed in literature, art, ...',
  'category': "['social science']",
  'year': 2002,
  'publisher': 'Oxford University Press',
  'rating': 0.0,
  'user_rating': 0.0}}

In [25]:
try:
    res = helpers.bulk( es , generator(result2))
    print("Working")
except Exception as e : 
    pass

c:\users\charalampos\appdata\local\programs\python\python39\lib\site-packages\elasticsearch\connection\base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
c:\users\charalampos\appdata\local\programs\python\python39\lib\site-packages\elasticsearch\connection\base.py:209: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


In [26]:
inp = input("Search Book: ")
myquery = {
   "_source":["title","year","author","rating","user_rating" ],
  "size":20,
  "track_scores":True,
  "query": {
    "match": {
      "title": inp
    }
  },
   "sort": [
       {"_script":
        {
      "type": "number",
      
      "script":{
        "lang": "painless", 
        "source":  "_score*3+doc['rating'].value+doc['user_rating'].value*2"
      },
      "order": "desc"
      }
    }
  ]
}


KeyboardInterrupt: Interrupted by user

In [ ]:
res = es.search (
    index='rbooks_elk',
    size= 10,
    body= myquery  
 )

In [ ]:
res

In [ ]:
r=res["hits"].get('hits')
for i in range(len(r)):
    for key in r[i]:
        if key == '_source': 
            print(i+1,":",r[i][key]['title'],'\n')